# What's on TV Tonight ?

This notebook groups every steps of the project :
- web scraping the french tv programme and get all the desired informations
    - channel number & channel name
    - programme name / title
    - beginning & duration time
    - rate /5 given by "TéléLoisirs"
    - synopsis of the programme
    
    
- display programmes by type (the user chooses the type of programme he wants to see), and sorted by rate for movies 


The code also works with what is on TV right now ("En ce moment" table) and the next day's porgramme ("Demain" table on the main webpage) for different TV guide (Orange, SFR, Bouygues etc.)

## Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## URL links & Constants

In [2]:
URL_TNT = "https://www.programme-tv.net/"
URL_CANAL = "https://www.programme-tv.net/programme/canal-plus/"
URL_NOW = "https://www.programme-tv.net/programme/en-ce-moment.html"
URL_TOMORROW = "https://www.programme-tv.net/programme/toutes-les-chaines/2020-12-23/" #change everyday

CHANNEL_NAME = "channel_name"
CHANNEL_NUMBER = "channel_number"
BEGINNING_HOUR = "beginning_hour"
PROGRAMME_TITLE = "programme_title"
PROGRAMME_TYPE = "programme_type" 
PROGRAMME_DURATION = "programme_duration"
SYNOPSIS = "synopsis"
RATE = "rate /5"

## Data retrieval

###### Get data for TNT (Télévision Numérique Terrestre) 

In [3]:
def loader_tnt(url_link: str) -> pd.DataFrame:
    
    # Set up scraping tool
    url_basic = url_link
    response_basic = requests.get(url_basic)
    soup = BeautifulSoup(response_basic.text, 'html.parser')
    info_box = soup.find_all("div", class_="doubleBroadcastCard")
    tv_programme = []
    
    for i in range(len(info_box)):
        # Get first useful informations on the main page
        channel_name    = info_box[i].find("a", class_="doubleBroadcastCard-channelName").text.strip()
        channel_number  = info_box[i].find("div", class_="doubleBroadcastCard-channelNumber").text.strip()
        beginning_hour  = info_box[i].find("div", class_="doubleBroadcastCard-hour").text.strip()
        programme_title = info_box[i].find("a", class_="doubleBroadcastCard-title").text.strip()
        programme_type  = info_box[i].find("div", class_="doubleBroadcastCard-type").text.strip()
        prgrm_duration  = info_box[i].find("span", class_="doubleBroadcastCard-durationContent").text.strip()
        url_link        = info_box[i].find("div", class_="doubleBroadcastCard-infos").a['href']

        # ReDo requests.get() to go through the link of the program to get new infos (synopsis / opinion) ...
        url_synopsis = url_link
        response_url_synopsis = requests.get(url_synopsis)
        soup_synopsis = BeautifulSoup(response_url_synopsis.text, 'html.parser')
        
        # ... based on different program type (movies, film, series) because the html structure is different
        movies_box = soup_synopsis.find_all("div", class_="programHome-mainContent")
        sport_box = soup_synopsis.find_all("div", class_="matchDetails")
        serie_box = soup_synopsis.find_all("div", class_="programCollectionEpisode-mainContent")
        
        # Create 3 variables of synopsis_type : it fills when condition is met and stays blank if not 
        # Get synopsis of film OR summary (mostly for sports, culture program, series & others)
        try:
            synopsis_film = (movies_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            synopsis_film = " "
            
        # Sports synopsis
        try:
            sport_details = (sport_box[0].find("p", class_="matchDetails-synopsis defaultStyleContentTags")
                            ).text.strip()
        except:
            sport_details = " "
      
        # Series synopsis 
        try:
            synopsis_serie = (serie_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                             ).text.strip()
        except:
            synopsis_serie = " "    
        
        # Get str(opinion) of "TéléLoisirs"
        try:
            opinion = movies_box[0].find("div", class_="review-loveText").text.strip()
        except:
            opinion = "Nothing"
            # I put "Nothing" that will be replace further because it may arrived that a movie has no rate -->
            # ... so I won't be able to sort by "RATE" if I have an empty str()

        info = [channel_name, programme_title, programme_type, channel_number, beginning_hour, prgrm_duration, 
                opinion, synopsis_film, sport_details, synopsis_serie]
        
        tv_programme.append(info)
       
    # Build a DataFrame from scraped data
    tv_programme = pd.DataFrame(tv_programme, columns=[CHANNEL_NAME, PROGRAMME_TITLE, PROGRAMME_TYPE, 
                                                       CHANNEL_NUMBER, BEGINNING_HOUR, PROGRAMME_DURATION, 
                                                       RATE, "synopsis_film", "sport_details", 
                                                       "synopsis_serie"])
    
    # Group all synopsis type (movies, sport, series) into one unique "synopsis" variable    
    tv_programme[SYNOPSIS] = (tv_programme["synopsis_film"] + tv_programme["sport_details"] 
                              + tv_programme["synopsis_serie"])
        
    # Replace str(opinion) by an integer representing the rate / 5 of the programme ...
    # couldn't scrape this variable on the website because the int is a <svg ...> (technical / knowledge issue)
    tv_programme[RATE].replace({"A ne pas manquer" : 5, "Très bon" : 4, "Bon" : 3, "Assez bon" : 2, 
                                "Décevant" : 1, "Nothing": 0}, inplace=True)

    # Drop intermediate columns
    tv_programme.drop(columns=["synopsis_film", "sport_details", "synopsis_serie"], axis=1, inplace=True)

    return tv_programme

In [4]:
programme_tnt = loader_tnt(URL_TNT)
# programme_tnt

###### Get data for "Le bouquet de Canal" (private channel - not necessary if you didn't subscribe) 

In [5]:
# Same function as loader_tnt with one additionnal command : lines 63-65
def loader_canal(url_link: str) -> pd.DataFrame:
    
    url_basic = url_link
    response_basic = requests.get(url_basic)
    soup = BeautifulSoup(response_basic.text, 'html.parser')
    info_box = soup.find_all("div", class_="doubleBroadcastCard")
    tv_programme = []
    
    for i in range(len(info_box)):
        channel_name    = info_box[i].find("a", class_="doubleBroadcastCard-channelName").text.strip()
        channel_number  = ""
        beginning_hour  = info_box[i].find("div", class_="doubleBroadcastCard-hour").text.strip()
        programme_title = info_box[i].find("a", class_="doubleBroadcastCard-title").text.strip()
        programme_type  = info_box[i].find("div", class_="doubleBroadcastCard-type").text.strip()
        prgrm_duration  = info_box[i].find("span", class_="doubleBroadcastCard-durationContent").text.strip()
        url_link        = info_box[i].find("div", class_="doubleBroadcastCard-infos").a['href']
        
        url_synopsis = url_link
        response_url_synopsis = requests.get(url_synopsis)
        soup_synopsis = BeautifulSoup(response_url_synopsis.text, 'html.parser')
        
        movies_box = soup_synopsis.find_all("div", class_="programHome-mainContent")
        sport_box = soup_synopsis.find_all("div", class_="matchDetails")
        serie_box = soup_synopsis.find_all("div", class_="programCollectionEpisode-mainContent")
        
        try:
            synopsis_film = (movies_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            synopsis_film = " "
            
        try:
            sport_details = (sport_box[0].find("p", class_="matchDetails-synopsis defaultStyleContentTags")
                            ).text.strip()
        except:
            sport_details = " "
      
        try:
            synopsis_serie = (serie_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                             ).text.strip()
        except:
            synopsis_serie = " "    
        
        try:
            opinion = movies_box[0].find("div", class_="review-loveText").text.strip()
        except:
            opinion = "Nothing"
        
        info = [channel_name, programme_title, programme_type, channel_number, beginning_hour, prgrm_duration, 
                opinion, synopsis_film, sport_details, synopsis_serie]
        
        tv_programme.append(info)
       
    tv_programme = pd.DataFrame(tv_programme, columns=[CHANNEL_NAME, PROGRAMME_TITLE, PROGRAMME_TYPE, 
                                                       CHANNEL_NUMBER, BEGINNING_HOUR, PROGRAMME_DURATION, 
                                                       RATE, "synopsis_film", "sport_details", 
                                                       "synopsis_serie"])
    
    tv_programme[SYNOPSIS] = (tv_programme["synopsis_film"] + tv_programme["sport_details"] 
                              + tv_programme["synopsis_serie"])
    
    # channel_number is not given by the website for canal channels --> enter values by hand (for my version)
    tv_programme[CHANNEL_NUMBER] = ["Chaîne n°4", "Chaîne n°40", "Chaîne n°41", "Chaîne n°42", 
                                    "Chaîne n°43", "Chaîne n°44"]
    
    tv_programme[RATE].replace({"A ne pas manquer" : 5, "Très bon" : 4, "Bon" : 3, "Assez bon" : 2, 
                                "Décevant" : 1, "Nothing": 0}, inplace=True)
    
    tv_programme.drop(columns=["synopsis_film", "sport_details", "synopsis_serie"], axis=1, inplace=True)
    
    return tv_programme

In [6]:
programme_canal = loader_canal(URL_CANAL)
# programme_canal

###### Concatenate the 2 dataframes (TNT + "Le bouquet de Canal")

In [7]:
def concatenate_frames(df_tnt: pd.DataFrame, df_canal: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([df_tnt, df_canal]).drop_duplicates(subset ="channel_name")

In [8]:
programme_tv = concatenate_frames(programme_tnt, programme_canal)
programme_tv

,channel_name,programme_title,programme_type,channel_number,beginning_hour,programme_duration,rate /5,synopsis
0,TF1,La grande incruste,Autre,Chaîne n°1,21h05,2h30min,0,"Tout au long de la soirée, l’animateur s’inc..."
1,France 2,Les comiques préférés des Français,Autre,Chaîne n°2,21h05,2h25min,0,"Pour les fe^tes, Laurence Boccolini a re´uni..."
2,France 3,Meurtres à Cognac,Téléfilm,Chaîne n°3,21h03,1h32min,3,Le maître de chai d'une grande maison de cogna...
3,Canal+,Manchester City / Newcastle,Sport,Chaîne n°4,21h00,1h57min,0,Bernardo Silva et les Cityzens remontent prog...
4,France 5,Echappées belles,Culture Infos,Chaîne n°5,20h50,1h35min,0,"À plus de 17000 km de Paris, au cœur de l'oc..."
5,M6,Belle et Sébastien 3 : le dernier chapitre,Cinéma,Chaîne n°6,21h05,1h40min,3,"En 1948, Sébastien, un garçon débrouillard de ..."
6,Arte,Au temps des cathédrales,Culture Infos,Chaîne n°7,20h50,55min,0,C’est en 1144 qu’est consacrée la basilique ...
7,C8,Le grand bêtisier,Autre,Chaîne n°8,21h05,1h57min,0,Que seraient les fêtes de fin d’année sans l...
8,W9,MacGyver,Série TV,Chaîne n°9,21h05,40min,0,L'équipe doit aider un baron de la drogue à ...
9,TMC,Columbo,Série TV,Chaîne n°10,21h05,1h50min,0,Freddy Brower gagne trente millions de dolla...


In [9]:
# programme_tv.to_csv("df_tv_programme.csv", index=True, sep="," , encoding="utf-8")

## Display best programmes by categorie

###### List of today's programme type ---> choose what you want to watch tonight 

In [10]:
programme_type_of_the_day = programme_tv[PROGRAMME_TYPE].unique().tolist()
programme_type_of_the_day

['Autre', 'Téléfilm', 'Sport', 'Culture Infos', 'Cinéma', 'Série TV']

###### Select a dataframe (TNT, Le bouquet de Canal or both) and the programme type you want to watch

In [11]:
def choose_programme_type(df: pd.DataFrame, programme_type: str=programme_type_of_the_day) -> pd.DataFrame:    
    pd.set_option("display.max_colwidth", -1)
    df_chosed = df.loc[df[PROGRAMME_TYPE] == programme_type]
    df_chosed = df_chosed.drop(columns=[PROGRAMME_TYPE]).set_index([CHANNEL_NAME, CHANNEL_NUMBER])
    return df_chosed.sort_values(by=RATE, ascending=False)#.head(3)

In [12]:
selected_type = choose_programme_type(programme_tv, "Cinéma")
selected_type

,,programme_title,beginning_hour,programme_duration,rate /5,synopsis
channel_name,channel_number,,,,,
Gulli,Chaîne n°18,Kirikou et les bêtes sauvages,21h00,1h25min,4,"En Afrique, un vieux sage conte les aventures de Kirikou, connu pour avoir triomphé de la sorcière Karaba. Parmi les autres exploits de l'enfant, il narre son combat contre une bête malfaisante qui ravageait chaque nuit le potager du village. Les villageois pensaient qu'un esprit dévastait leurs cultures. Kirikou, une nuit, se cache et observe. Il aperçoit un animal venu de la forêt et décide de construire des pièges."
M6,Chaîne n°6,Belle et Sébastien 3 : le dernier chapitre,21h05,1h40min,3,"En 1948, Sébastien, un garçon débrouillard de 12 ans, vit heureux dans les Alpes françaises avec César, son grand-père. L'enfant aime faire l'école buissonnière avec sa chienne Belle, désormais mère de trois chiots adorables mais turbulents. Quand Joseph, l'ancien maître de Belle, surgit un jour pour la récupérer, l'enfant décide de se rendre de l'autre côté de la montagne pour protéger son amie et ses petits. Mais Joseph réussit à les retrouver et enlève Belle et ses chiots. Sébastien et César se lancent alors à la poursuite de cet homme sans scrupules."
La Chaîne parlementaire,Chaîne n°13,Le bon plaisir,20h59,1h30min,3,"Un soir, Claire se fait voler son sac. A l'intérieur, il y a une lettre du président de la République, avec qui elle a eu une liaison. Le ministre de l'Intérieur, ami de longue date, prend l'affaire en main et demande à ses services de retrouver le voleur. Ce dernier, Pierre, ne s'aperçoit pas immédiatement de l'existence de la missive, contrairement à son ami Herbert, journaliste politique."
France 4,Chaîne n°14,Pup Star,21h10,1h20min,3,"Les aventures de Tiny, un Yorkshire Terrier participant à un concours de chant télévisé et rêvant de devenir célèbre, qui se retrouve séparée de son propriétaire par un employé de la fourrière. La chienne doit alors retrouver son maître avec l'aide d'une nouvelle amie, par le chemin des ""musichiens""."
Canal+ Cinéma,Chaîne n°40,Cold Blood Legacy : la mémoire du sang,20h55,1h29min,2,"De nos jours, dans les montagnes enneigées de l'Etat de Washington, Melody fait une chute spectaculaire alors qu'elle sillonnait la région à grande vitesse en motoneige. La jeune femme, blessée à la jambe, parvient à rejoindre la cabane isolée où vit reclus Henry, un célèbre tueur à gages, résolu depuis longtemps à s'éloigner de son milieu. L'homme se souvient alors de son existence, dix mois plus tôt, à New York, quand il était chargé, contre d'importantes sommes d'argent, d'éliminer des cibles très protégées"
Canal+ Family,Chaîne n°44,Le prince oublié,20h53,1h39min,2,"Sofia, 8 ans, est élevée par son père très aimant. L'homme, veuf, qui travaille dans un garage, passe ses journées à peaufiner les moindres détails des histoires qu'il racontera le soir à sa fille. En effet, chaque soir, il raconte à l'enfant l'histoire feuilletonnante d'un prince et de sa princesse, sans cesse contrariés par le méchant Pritprout. Et quand Sofia s'endort, l'histoire s'interrompt et le prince, un comédien, se promène dans le décor du conte, discutant avec acteurs, figurants et techniciens."


## Options for the dropdowns to choose (for app.py)

In [13]:
guide_tv = {
    "basic_tonight"       : "https://www.programme-tv.net",
    "basic_now"           : "https://www.programme-tv.net/programme/en-ce-moment.html",
    "belgian_tonight"     : "https://www.programme-tv.net/programme/proximus-18/",
    "belgian_nom"         : "https://www.programme-tv.net/programme/proximus-18/en-ce-moment.html",
    "orange_tonight"      : "https://www.programme-tv.net/programme/orange-12/",
    "orange_now"          : "https://www.programme-tv.net/programme/orange-12/en-ce-moment.html",
    "sfr_tonight"         : "https://www.programme-tv.net/programme/sfr-25/",
    "sfr_now"             : "https://www.programme-tv.net/programme/sfr-25/en-ce-moment.html",
    "free_tonight"        : "https://www.programme-tv.net/programme/free-13/",
    "free_now"            : "https://www.programme-tv.net/programme/free-13/en-ce-moment.html",
    "numericable_tonight" : "https://www.programme-tv.net/programme/numericable-7/",
    "numericable_now"     : "https://www.programme-tv.net/programme/numericable-7/en-ce-moment.html",
    "bouygues_tonight"    : "https://www.programme-tv.net/programme/bouygues-24/",
    "bouygues_now"        : "https://www.programme-tv.net/programme/bouygues-24/en-ce-moment.html",
    "canalsat_tonight"    : "https://www.programme-tv.net/programme/canal-5/",
    "canalsat_now"        : "https://www.programme-tv.net/programme/canal-5/en-ce-moment.html",
    "tnt_tonight"         : "https://www.programme-tv.net/programme/programme-tnt.html",
    "tnt_now"             : "https://www.programme-tv.net/programme/programme-tnt/en-ce-moment.html",
    "canal_+_tonight"     : "https://www.programme-tv.net/programme/canal-plus/",
    "canal_+_now"         : "https://www.programme-tv.net/programme/canal-plus/en-ce-moment.html"
}

In [14]:
guide = ["basic", "belgian", "orange", "sfr", "free", "numericable", "bouygues", "canalsat", "canal +", 
         "tnt", "canal sat"]